Check of 2x6 Wood Joist Design per O86-09  
E.Durham - 16-Aug-2018

In [1]:
import pint
unit = pint.UnitRegistry(system='mks')
Q = unit.Quantity

# define synonyms for common units
inch = unit.inch; mm = unit.mm; m = unit.m; kPa = unit.kPa; MPa = unit.MPa; 
psi = unit.psi; kN = unit.kN; N = unit.N; ksi = unit.ksi; 
dimensionless = unit.dimensionless; s = unit.second; kg = unit.kg

In [2]:
# Dead Load
DL_2x4 = 50 * (N/m**2) + 30 * (N/m**2)  # 400 and 600 on-centre, from Wood 
# Design Manual Table 11.26
DL_2x6 = 50 * (N/m**2)  # 600 on-centre, ditto
DL_19 = 80 * (N/m**2)  # 19 mm S-P-F sheathing, from Wood Design Manual 
# Table 11.27
DL = DL_2x4 + DL_2x6 + DL_19
DL.ito(kN/m**2)
print('DL =', round(DL,2))

DL = 0.21 kilonewton / meter ** 2


In [3]:
# Load Case
L = 2.8 * m # span of 2x6 joist, per 629768-S-A2105-ANN-0211 Rev_01, 
# verified 15-Aug-2018 by EWD
LL = 2 * (kN/m**2) # Given Live Load
Tributary_width = 0.6 * m
sf_DL = 1.25
sf_LL = 1.5
w = ((sf_DL * DL)+(sf_LL * LL)) * Tributary_width # factored uniform load
w.ito(kN/m)
print('w =', round(w,2))

w = 1.96 kilonewton / meter


In [4]:
# Bending
M_max = (w * L**2) / 8
M_max.ito(kN*m)
print('M_max =', round(M_max,2))

M_max = 1.92 kilonewton * meter


In [5]:
# Trial Member Properties from Wood Design Manual 2010 Table 11.5
# Try 2x6 S-P-F No.1/No.2
b = 38 * mm
d = 140 * mm
A = 5.32 * 10**3 * mm**2 # Area
S_x = 124 * 10**3 * mm**3 # Section Modulus
I_x = 8.66 * 10**6 * mm**4 # Second Moment of Area

O86-09 5.5.4 Bending moment resistance  
5.5.4.1 General  
The factored bending moment resistance, $ M_r $ of sawn members shall be taken as follows:  
$ M_r = \phi F_b S K_{Zb} K_L $  
where  
$ \phi = 0.9 $  
$ F_b = f_b(K_D K_H K_{Sb} K_T) $  
where  
$ f_b = $ specified strength in bending, MPa  
$ K_{Zb} = $ size factor in bending  
$ K_L = $ lateral stability factor  

In [6]:
# Determine appropriate value for Lateral Stability Factor, K_L per 5.5.4.2
print('Depth to width ratio is: ', round(d/b,2) )

Depth to width ratio is:  3.68 dimensionless


Per 5.5.4.2.1(a) depth to width ratio less than 4:1 do not require intermediate lateral supports and K_L may be taken as unity. Lateral restraint is required at beam ends.

In [7]:
K_H = 1.0 * dimensionless  # System Factor per 5.4.4
K_L = 1.0 * dimensionless  # Lateral stability factor per 5.5.4.2
K_D = 1.15 * dimensionless # Load duration factor per Table 4.3.2.2 short-term
K_Sb = 1.0 * dimensionless  # Service condition factor per Table 5.4.2
K_T = 1.0 * dimensionless  # Treatment factor per Table 5.4.3 untreated
K_Zb = 1.4  # from Table 5,4,5
f_b = 11.8 * MPa  # S-P-F No.1/No.2 from Table 5.3.1A

In [8]:
F_b = f_b*(K_D*K_H*K_Sb*K_T)
F_b.ito(MPa)
print('F_b =', round(F_b,2))

F_b = 13.57 megapascal


In [9]:
phi = 0.9 * dimensionless
M_r = phi*F_b*S_x*K_Zb*K_L
M_r.ito(kN*m)
print('M_r =', round(M_r,2))

M_r = 2.12 kilonewton * meter


In [10]:
M_r > M_max

True

Factored bending resistance is greater than factored load effects therefore, OK for Bending

## Check Bearing per 5.5.7  
The factored compressive resistance perpendicular to grain under the effect of all factored applied loads shall be taken as $ Q_r $ as follow:  
$ Q_r = \phi F_{cp} A_b K_B K_{Zcp} $  
where  
$ \phi = 0.8 $  
$ F_{cp} = f_{cp} (K_D K_{Scp} K_T) $  
where  
$ f_{cp} = $ specified strength in compression perpendicular to grain, MPa  
$ A_b = $ bearing area, mm^2  
$ K_B = $ length of bearing factor (Clause 5.5.7.6)  
$ K_{Zcp} = $ size factor for bearing (Clause 5.5.7.5)  

In [11]:
Q_f = (w*L)/2
Q_f.ito(kN)
print('Q_f =', round(Q_f,2))

Q_f = 2.74 kilonewton


In [12]:
A_b = (38 * mm) * (20 * mm)  # ASSUMED, need to verify thickness of steel 
# supports
phi = 0.8 * dimensionless
f_cp = 5.3 * MPa  # S-P-F No.1/No.2 from Table 5.3.1A
K_D = 1.15 * dimensionless  # Load duration factor per Table 4.3.2.2 
# short-term
K_B = 1.0 * dimensionless  # Length of bearing factor per Table 5.5.7.6 unity
# b/c < 75mm from end
K_Scp = 1.0 * dimensionless  # Service condition factor per Table 5.4.2 for 
# dry service conditions
K_Zcp = 1.0 * dimensionless  # Size factor for bearing per 5.5.7.5 
# 38/140 << 1.0

In [13]:
F_cp = f_cp*(K_D*K_Scp*K_T)
F_cp.ito(MPa)
print('F_cp =', round(F_cp,2))

F_cp = 6.09 megapascal


In [14]:
Q_r = phi*F_cp*A_b*K_B*K_Zcp
Q_r.ito(kN)
print('Q_r =', round(Q_r,2))

Q_r = 3.71 kilonewton


In [15]:
Q_r > Q_f

True

Compressive resistance perpendicular to grain exceeds factored bearing forces; therefore, OK for Bearing

## Check Shear

5.5.5 Shear Resistance  
5.5.5.1 General  
The factored shear resistance, $ V_r $ of sawn members shall be taken as follows:  
$ V_r = \phi F_v \frac{2 A_n}{3} K_{Zv} $  
where  
$ \phi = 0.9 $  
$ F_v = f_v (K_D K_H K_{Sv} K_T) $  
where  
$ f_v = $ specified shrength in shear, MPa (Clause 5.3)  
$ A_n = $ net area of cross-section, mm^2 (Clause 4.3.8)  
$ K_{Zv} = $ size factor in shear (Clause 5.4.5)  

In [16]:
V_f = (w*L)/2
print('V_f =', round(V_f,2))

V_f = 2.74 kilonewton


In [17]:
phi = 0.9
A_n = A
K_D = 1.15 * dimensionless  # Load duration factor per Table 4.3.2.2 
# short-term
K_H = 1.0 * dimensionless  # System Factor per 5.4.4
K_Sv = 1.0 * dimensionless  # Service condition factor per Table 5.4.2 dry
K_T = 1.0 * dimensionless  # Treatment factor per Table 5.4.3 untreated
K_Zv = 1.4 * dimensionless  # from Table 5,4,5
f_v = 1.5 * MPa  # S-P-F No.1/No.2 from Table 5.3.1A
F_v = f_v*(K_D*K_H*K_Sv*K_T)
F_v.ito(MPa)
print('F_v =', round(F_v,2))

F_v = 1.72 megapascal


In [18]:
V_r = phi*F_v*((2*A_n)/3)*K_Zv
V_r.ito(kN)
print('V_r =', round(V_r,2))

V_r = 7.71 kilonewton


In [19]:
V_r > V_f

True

Factored shear resistance is greater than factored shear; therefore, OK for Shear

## Check Deflection  
Load combination for serviceability limit state 1.0DL + 1.0LL  

In [20]:
w = (DL + LL) * Tributary_width
w.ito(kN/m)
print('w =', round(w,2))

w = 1.33 kilonewton / meter


In [21]:
delta_limit = L/180
delta_limit.ito(mm)
print('L/180 =', round(delta_limit,1))

L/180 = 15.6 millimeter


In [22]:
E = 9500 * MPa  # Modulus of Elasticity per Table 5.3.1A S-P-F No.1/No.2
K_SE = 1.0 * dimensionless  # Service condition factor per Table 5.4.2 dry
K_T = 1.0 * dimensionless  # Treatment factor per Table 5.4.3 untreated
E_S = E*(K_SE*K_T)  # per 4.5.1
E_S.ito(MPa)
E_S

9500.0 <Unit('megapascal')>

In [23]:
delta_max = (5*w*L**4)/(384*E_S*I_x)
delta_max.ito(mm)
print('Max deflection =', round(delta_max,1))

Max deflection = 12.9 millimeter


In [24]:
delta_limit > delta_max

True

Elastic deflection limit is greater than elastic deflection due to service loading; therefore, OK for Deflection